In [1]:
infile = "day15.txt"

In [2]:
import numpy as np
with open(infile) as fd:
    grid = np.array([[int(ch) for ch in line.rstrip()] for line in fd], np.int32)

In [3]:
from queue import PriorityQueue
from collections import defaultdict

neighbors = [(-1, 0), (0, -1), (1, 0), (0, 1)]

def trace_back(grid, path, curr):
    tr = np.ndarray(grid.shape, dtype='U1')
    tr[:] = " "
    tr[curr] = "#"
    while curr in path:
        curr = path[curr]
        tr[curr] = '#'
    return tr

def a_star(start, end, grid):
    path = {}
    cheapest = defaultdict(lambda: float('inf'))
    cheapest[start] = 0
    q = PriorityQueue()
    q.put((0, start))
    seen = set()
    seen.add(start)
    while not q.empty():
        _, curr = q.get()
        seen.remove(curr)
        if curr == end:
            return (cheapest[curr], trace_back(grid, path, curr))
        for dx, dy in neighbors:
            n = (curr[0]+dx, curr[1]+dy)
            if not (0 <= n[0] < grid.shape[0] and 0 <= n[1] < grid.shape[1]):
                continue
            cost = cheapest[curr] + grid[n]
            if cost < cheapest[n]:
                path[n] = curr
                cheapest[n] = cost
                if n not in seen:
                    q.put((cost, n))
                    seen.add(n)

In [4]:
start = (0, 0)
end = (grid.shape[0]-1, grid.shape[1]-1)
a_star(start, end, grid)

(583,
 array([['#', '#', '#', ..., ' ', ' ', ' '],
        [' ', ' ', ' ', ..., ' ', ' ', ' '],
        [' ', ' ', ' ', ..., ' ', ' ', ' '],
        ...,
        [' ', ' ', ' ', ..., ' ', ' ', ' '],
        [' ', ' ', ' ', ..., '#', '#', ' '],
        [' ', ' ', ' ', ..., ' ', '#', '#']], dtype='<U1'))

In [5]:
def _rot(n):
    return n % 9 + 1

rot = np.vectorize(_rot)

def expand(grid):
    dim = 5
    full_grid = [[None for _ in range(dim)] for _ in range(dim)]
    full_grid[0][0] = grid
    for n in range(1, dim):
        grid = rot(grid)
        for x in range(n + 1):
            #print(n)
            y = n - x
            full_grid[x][y] = grid
    for n in range(1, dim):
        grid = rot(grid)
        for x in range(n, dim):
            #print(n+dim-1)
            y = dim - 1 + n - x
            full_grid[x][y] = grid
    return np.block(full_grid)

In [6]:
full_grid = expand(grid)

In [7]:
start = (0, 0)
end = (full_grid.shape[0]-1, full_grid.shape[1]-1)
min_cost, path = a_star(start, end, full_grid)

In [8]:
#print('\n'.join(''.join(row) for row in path))